In [84]:
import os
import sys
import time
import numpy as np
import pandas as pd
from dateutil import tz
from scipy import fftpack, stats
from itertools import chain
from datetime import datetime,timedelta

In [12]:
path = "F:/DATA/hope/1s5wlcm6/accelerometer/2018-01-20 17_00_00.csv"
data = pd.read_csv(path)
data.head(10)

,timestamp,UTC time,accuracy,x,y,z
0,1516467868973,2018-01-20T17:04:28.973,3,-0.186753,5.569083,7.245076
1,1516467869113,2018-01-20T17:04:29.113,3,-0.383084,5.674431,7.292961
2,1516467869133,2018-01-20T17:04:29.133,3,-0.565049,5.944984,7.723930
3,1516467869157,2018-01-20T17:04:29.157,3,-0.536318,5.892310,7.829279
4,1516467869158,2018-01-20T17:04:29.158,3,-0.837996,5.937801,8.032792
5,1516467869160,2018-01-20T17:04:29.160,3,-0.677580,5.719923,8.154900
6,1516467869175,2018-01-20T17:04:29.175,3,-0.399844,5.478101,7.917867
7,1516467869315,2018-01-20T17:04:29.315,3,-0.232245,5.310502,7.728719
8,1516467869325,2018-01-20T17:04:29.325,3,-0.395055,5.523592,8.097438
9,1516467869333,2018-01-20T17:04:29.333,3,-0.660820,5.494861,8.209969


In [76]:
## 1. smooth to a certain Hz
hz = 10
def smooth_data(data,hz):
  stamp0 = datetime.fromtimestamp(data['timestamp'][0]/1000)
  stamp1 = [stamp0.year,stamp0.month,stamp0.day,stamp0.hour]
  t = np.array(data["timestamp"])
  x = np.array(data["x"])
  y = np.array(data["y"])
  z = np.array(data["z"])
  mag = np.sqrt(x**2+y**2+z**2)
  t_diff = t[1:]-t[:-1]
  t_active = sum(t_diff[t_diff<5*1000])
  t_active = t_active/1000/60  ## in minute
  a = np.floor((t - min(t))/(1/hz*1000))  ## bin
  b = []
  for i in np.unique(a):
    index = a==i
    b.append(np.mean(mag[index]))
  b = np.array(b)
  return stamp1,t_active,np.unique(a)*(1/hz*1000),b

In [14]:
stamp,t_active,t,mag = smooth_data(data,hz)

In [19]:
## 2. step estimation funciton
q = 75
c = 1.05
def step_est(t,mag,t_active,q,c):
  if np.mean(mag)>8:
    g = 9.8
  else:
    g = 1
  h = max(np.percentile(mag,q),c*g)
  step = 0
  current = -350
  for j in range(len(t)):
    if(mag[j]>=h and t[j]>=current+350):
      step = step + 1
      current = t[j]
  final_step = int(step/t_active*60)
  return final_step

In [20]:
step_est(t,mag,t_active,q,c)

151

In [70]:
## 3. other statistics from accelerometer data
def acc_stats(mag,hz):
  m_mag = np.mean(mag)
  sd_mag = np.std(mag)
  X = fftpack.fft(mag)
  remove_X = X[X!=max(X)]
  f_vec = fftpack.fftfreq(len(mag))*hz
  remove_F = f_vec[X!=max(X)]
  freq = remove_F[remove_X==max(remove_X)][0]
  amplitude_spectrum = np.abs(X)/hz
  eg = sum(amplitude_spectrum**2)*hz/len(mag)**2
  entropy = stats.entropy(mag)
  return [m_mag,sd_mag,freq,eg,entropy]

In [71]:
others = acc_stats(mag,hz)
others

[9.90077849773792,
 0.8550554806154826,
 1.8614130434782608,
 9.875653473620146,
 8.677132504256651]

In [91]:
## wrap up every thing
hz = 10; q = 75; c = 1.05
def GetAccStats(data,hz,q,c):
  stamp,t_active,t,mag = smooth_data(data,hz)
  steps = step_est(t,mag,t_active,q,c)
  others = others = acc_stats(mag,hz)
  result = [stamp[0],stamp[1],stamp[2],stamp[3],t_active,steps,
            others[0],others[1],others[2],others[3],others[4]]
  return np.array(result)

In [92]:
GetAccStats(data,hz,q,c)
## year,month,day,hour,active_min,steps,mean_mag,sd_mag,freq,energy,entropy

array([2.01800000e+03, 1.00000000e+00, 2.00000000e+01, 1.20000000e+01,
       9.88225000e+00, 1.85700000e+03, 9.90077850e+00, 8.55055481e-01,
       1.86141304e+00, 9.87565347e+00, 8.67713250e+00])